# Facial Expression Recognition Using CNN

# Project Goal:
**The primary goal of DeepFER:** Facial Emotion Recognition Using Deep Learning is to develop an advanced and efficient system capable of accurately identifying and classifying human emotions from facial expressions in real-time. By leveraging state-of-the-art Convolutional Neural Networks (CNNs) and Transfer Learning techniques, this project aims to create a robust model that can handle the inherent variability in facial expressions and diverse image conditions. The system will be trained on a comprehensive dataset featuring seven distinct emotions: angry, sad, happy, fear, neutral, disgust, and surprise. The ultimate objective is to achieve high accuracy and reliability, making DeepFER suitable for applications in human-computer interaction, mental health monitoring, customer service, and beyond. Through this project, we aim to bridge the gap between cutting-edge AI research and practical emotion recognition applications, contributing to more empathetic and responsive machine interactions with humans.

Emotion Classes:
* Angry: Images depicting expressions of anger.
* Sad: Images depicting expressions of sadness.
* Happy: Images depicting expressions of happiness.
* Fear: Images depicting expressions of fear.
* Neutral: Images depicting neutral, non-expressive faces.
* Disgust: Images depicting expressions of disgust.
* Surprise: Images depicting expressions of surprise.

### **Import Libraries**

This section imports the essential libraries needed for constructing and training a convolutional neural network (CNN) for facial expression recognition.

- `os`: Provides functions to interact with the operating system, useful for handling file operations.
- `cv2`: OpenCV library for computer vision, used here for processing images.
- `numpy`: A library for numerical computing, essential for array manipulations.
- `tensorflow`: The TensorFlow library used for deep learning tasks.
- `train_test_split` from `sklearn.model_selection`: Splits the dataset into training and testing subsets.
- `ImageDataGenerator` from `tensorflow.keras.preprocessing.image`: Generates batches of augmented data for training.
- `LabelEncoder` from `sklearn.preprocessing`: Converts categorical labels into numerical format.
- `to_categorical` from `keras.utils`: Transforms class labels into a binary class matrix.
- `Sequential` from `keras.models`: A linear stack of layers used to build deep learning models.
- `Dense`, `Conv2D`, `Dropout`, `BatchNormalization`, `MaxPooling2D`, `Flatten` from `keras.layers`: Various layers used in the CNN architecture.
- Optimizers (`Adam`, `RMSprop`, `SGD`) from `keras.optimizers`: Algorithms that adjust model weights during training.
- `plt` from `matplotlib.pyplot`: A plotting library for visualizing training and validation curves.
- Callbacks (`ModelCheckpoint`, `EarlyStopping`, `ReduceLROnPlateau`) from `keras.callbacks`: Tools used during training to enhance model performance or handle interruptions.


In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import gradio as gr

# Defining the path and classes

In [2]:
# Define the base path to your dataset
BASE_PATH = r"Face Emotion Recognition Dataset"
TRAIN_PATH = os.path.join(BASE_PATH, "train")
VALIDATION_PATH = os.path.join(BASE_PATH, "validation")

# Define emotion classes
EMOTION_CLASSES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Image parameters
IMG_SIZE = 48  # Standard size for emotion recognition
CHANNELS = 1   # Grayscale images

In [3]:
# Load training dataset
# Initialize lists for training data
train_images = []
train_labels = []

print("="*50)
print("LOADING TRAINING DATA")
print("="*50)

for emotion_class in EMOTION_CLASSES:
    emotion_path = os.path.join(TRAIN_PATH, emotion_class)
    
    if not os.path.exists(emotion_path):
        print(f"Warning: Path {emotion_path} does not exist!")
        continue
        
    print(f"Loading {emotion_class} images...")
    emotion_images = os.listdir(emotion_path)
    
    # Loop through each image in the emotion folder
    for image_name in emotion_images:
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(emotion_path, image_name)
            
            # Load image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if img is not None:
                # Resize image to target size
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                
                # Normalize pixel values to [0, 1]
                img = img.astype('float32') / 255.0
                
                # Add to lists
                train_images.append(img)
                train_labels.append(emotion_class)
    
    print(f"Loaded {len([l for l in train_labels if l == emotion_class])} {emotion_class} images")

LOADING TRAINING DATA
Loading angry images...
Loaded 3993 angry images
Loading disgust images...
Loaded 436 disgust images
Loading fear images...
Loaded 4103 fear images
Loading happy images...
Loaded 7164 happy images
Loading neutral images...
Loaded 4982 neutral images
Loading sad images...
Loaded 4938 sad images
Loading surprise images...
Loaded 3205 surprise images


In [4]:
# Convert to numpy arrays
X_train = np.array(train_images)
y_train = np.array(train_labels)

print(f"\nTraining data loaded!")
print(f"Training images shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")


Training data loaded!
Training images shape: (28821, 48, 48)
Training labels shape: (28821,)


In [5]:
# Initialize lists for validation data
val_images = []
val_labels = []

print("\n" + "="*50)
print("LOADING VALIDATION DATA")
print("="*50)

# Loop through each emotion class in validation data
for emotion_class in EMOTION_CLASSES:
    emotion_path = os.path.join(VALIDATION_PATH, emotion_class)
    
    if not os.path.exists(emotion_path):
        print(f"Warning: Path {emotion_path} does not exist!")
        continue
        
    print(f"Loading {emotion_class} images...")
    emotion_images = os.listdir(emotion_path)
    
    # Loop through each image in the emotion folder
    for image_name in emotion_images:
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(emotion_path, image_name)
            
            # Load image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if img is not None:
                # Resize image to target size
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                
                # Normalize pixel values to [0, 1]
                img = img.astype('float32') / 255.0
                
                # Add to lists
                val_images.append(img)
                val_labels.append(emotion_class)
    
    print(f"Loaded {len([l for l in val_labels if l == emotion_class])} {emotion_class} images")

# Convert to numpy arrays
X_val = np.array(val_images)
y_val = np.array(val_labels)

print(f"\nValidation data loaded!")
print(f"Validation images shape: {X_val.shape}")
print(f"Validation labels shape: {y_val.shape}")


LOADING VALIDATION DATA
Loading angry images...
Loaded 960 angry images
Loading disgust images...
Loaded 111 disgust images
Loading fear images...
Loaded 1018 fear images
Loading happy images...
Loaded 1825 happy images
Loading neutral images...
Loaded 1216 neutral images
Loading sad images...
Loaded 1139 sad images
Loading surprise images...
Loaded 797 surprise images

Validation data loaded!
Validation images shape: (7066, 48, 48)
Validation labels shape: (7066,)


In [6]:
# Reshape images to add channel dimension for CNN
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)
X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)

print("Images reshaped for CNN:")
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Images reshaped for CNN:
Training data shape: (28821, 48, 48, 1)
Validation data shape: (7066, 48, 48, 1)


In [7]:
# Reshape images to add channel dimension for CNN
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)
X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)

print("Images reshaped for CNN:")
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Images reshaped for CNN:
Training data shape: (28821, 48, 48, 1)
Validation data shape: (7066, 48, 48, 1)
